<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz.

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastian Aveldaño
- Nombre de alumno 2: Felipe Urrutia


### **Link de repositorio de GitHub:** https://github.com/furrutiav/lab-mds-2022

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 23/06/2022
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

  Attempting uninstall: plotly
    Found existing installation: plotly 5.7.0
    Uninstalling plotly-5.7.0:
      Successfully uninstalled plotly-5.7.0


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\felip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [6]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [7]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [8]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\felip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [9]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [11]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

### Obtener a traves de Bag of Words (CountVectorizer) caracteristicas del resumen de historia de cada personaje.

In [49]:
bog = CountVectorizer(
    tokenizer= StemmerTokenizer(),
    ngram_range=(1,2),
#     max_features=1000
)

In [50]:
bow_features = bog.fit_transform(df_comics["history_text"])
bog_feature_names = bog.get_feature_names_out()

In [51]:
df_bow = pd.DataFrame(bow_features.toarray(), columns=bog_feature_names)
df_bow

,!,! !,! '',! (,! ),"! ,",! --,! .,! ``,! alison,...,�kick� .,�kick� and,�polic,�polic man�,�white,�white room�,�ǣmortal,�ǣmortal instruments���,��wors,"��wors ,"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1281,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1283,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Aplicar MinMaxScaler sobre los atributos de interes.

In [52]:
atributos_de_interes = [
    'intelligence_score', 
    'strength_score', 
    'speed_score', 
    'durability_score', 
    'power_score', 
    'combat_score'
]

ct = ColumnTransformer([
    ("MinMax", MinMaxScaler(), atributos_de_interes),
    ("BOW", bog, "history_text")
])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

### Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador MultinomialNB().

In [87]:
nb_pipe = Pipeline(
    steps=[
        ('preprocessing', ct),
        ("selector", SelectPercentile(f_classif, percentile=90)),
        ("clf", MultinomialNB())
    ]
)

In [73]:
df_comics["alignment"].value_counts()

Good       743
Bad        429
Neutral    113
Name: alignment, dtype: int64

In [74]:
X, y = df_comics.drop(columns="alignment"), df_comics["alignment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=69)

### Entrenar el pipeline, calcular el classification_report asociado y comentar los resultados.

In [75]:
nb_pipe.fit(X_train, y_train)
y_pred = nb_pipe.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.67      0.09      0.16       134
        Good       0.59      0.98      0.73       220
     Neutral       0.00      0.00      0.00        32

    accuracy                           0.59       386
   macro avg       0.42      0.36      0.30       386
weighted avg       0.57      0.59      0.47       386



C:\Users\felip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\felip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\felip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Entrenar un DummyClassifier con estrategia statified, calcular el classification_report asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

In [77]:
from sklearn.dummy import DummyClassifier

In [78]:
dummy_pipe = Pipeline(
    steps=[
        ('preprocessing', ct),
        ("selector", SelectPercentile(f_classif, percentile=90)),
        ("dummy", DummyClassifier(strategy="stratified"))
    ]
)

In [79]:
dummy_pipe.fit(X_train, y_train)
y_pred = dummy_pipe.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.39      0.37      0.38       134
        Good       0.59      0.59      0.59       220
     Neutral       0.07      0.09      0.08        32

    accuracy                           0.47       386
   macro avg       0.35      0.35      0.35       386
weighted avg       0.48      0.47      0.47       386



```
Al observar el macro avg del f1-score, se puede apreciar que el modelo del dummy es superior al modelo con el clasificador Naive Bayes. Esto es una señal de alerta de que el clasificador no es adecuado para el problema, dado que una clasificación aleatoria, como es el dummy, obtiene mejor desempeño que el clasificador elejido. Por lo tanto, NB no sólo no está aportando ninguna información nueva, si no que está informando comportamientos erróneos. Con esto, no recomendamos la utilizacion del modelo NB.
```

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [88]:
nb_pipe.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                    ['intelligence_score', 'strength_score',
                                     'speed_score', 'durability_score',
                                     'power_score', 'combat_score']),
                                   ('BOW',
                                    CountVectorizer(ngram_range=(1, 2),
                                                    tokenizer=<__main__.StemmerTokenizer object at 0x00000189C6F9A520>),
                                    'history_text')])),
  ('selector', SelectPercentile(percentile=90)),
  ('clf', MultinomialNB())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                  ['intelligence_score', 'strength_score',
                                   'speed_score', 'durability_score',
                                   'power_score', 'combat_scor

In [85]:
from sklearn.neighbors import KNeighborsClassifier

In [107]:
param_grid = [
    # grilla 1: tree
    {
        "selector__percentile": [20, 40, 60, 80],
        "clf": [DecisionTreeClassifier(random_state=42)],
        "clf__criterion": ["gini", "entropy"],
        "preprocessing__BOW__ngram_range": [(1,1), (1,2), (1,3)],
        'preprocessing__BOW__min_df': [1, 5, 10],
        'preprocessing__BOW__max_features': [1000, None]
    },
    # grilla 2: knn
    {
        "selector__percentile": [20, 40, 60, 80],
        "clf": [KNeighborsClassifier()],
        "clf__n_neighbors": [2, 4, 5, 10],
        "preprocessing__BOW__ngram_range": [(1,1), (1,2), (1,3)],
        'preprocessing__BOW__min_df': [1, 5, 10],
        'preprocessing__BOW__max_features': [1000, None]
    },
    # grilla 3: random forest
    {
        "clf": [RandomForestClassifier(random_state=42)],
        "clf__criterion": ["gini", "entropy"],
        "clf__bootstrap": [True, False],
        "preprocessing__BOW__ngram_range": [(1,1), (1,2), (1,3)],
        'preprocessing__BOW__min_df': [1, 5, 10],
        'preprocessing__BOW__max_features': [1000, None]
    },
]

In [105]:
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

In [112]:
hgs = HalvingGridSearchCV(nb_pipe, param_grid, n_jobs=-1, scoring='f1_macro', verbose=10)

In [113]:
hgs.fit(X, y)

n_iterations: 4
n_required_iterations: 6
n_possible_iterations: 4
min_resources_: 30
max_resources_: 1285
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 504
n_resources: 30
Fitting 5 folds for each of 504 candidates, totalling 2520 fits
----------
iter: 1
n_candidates: 168
n_resources: 90
Fitting 5 folds for each of 168 candidates, totalling 840 fits
----------
iter: 2
n_candidates: 56
n_resources: 270
Fitting 5 folds for each of 56 candidates, totalling 280 fits
----------
iter: 3
n_candidates: 19
n_resources: 810
Fitting 5 folds for each of 19 candidates, totalling 95 fits


HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                               ColumnTransformer(transformers=[('MinMax',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                 'strength_score',
                                                                                 'speed_score',
                                                                                 'durability_score',
                                                                                 'power_score',
                                                                                 'combat_score']),
                                                                               ('BOW',
                                                                     

In [114]:
hgs.best_score_

0.3948548550180017

In [119]:
best_pipe = hgs.best_estimator_
best_pipe.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                    ['intelligence_score', 'strength_score',
                                     'speed_score', 'durability_score',
                                     'power_score', 'combat_score']),
                                   ('BOW',
                                    CountVectorizer(ngram_range=(1, 2),
                                                    tokenizer=<__main__.StemmerTokenizer object at 0x0000018B47F0AEE0>),
                                    'history_text')])),
  ('selector', SelectPercentile(percentile=40)),
  ('clf', DecisionTreeClassifier(random_state=42))],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                  ['intelligence_score', 'strength_score',
                                   'speed_score', 'durability_score',
                                   'po

```
Se evaluaron 3 clasificadores: "Decision Tree", "KNN" y "Random Forest". La grilla de cada uno incluye el porcentaje de columnas seleccionadas, varioaciones de 1 o 2 hiperparámetros de cada clasificador, y 3 tipos de variaciones para Count Vectorizer, en los parámetros "rango de n-gramas", "min_df" (frecuencia mínima de documentos), y "max_features" (limitar la cantidad de elementos distintos presentados, reteniendo los más frecuentes). 
 
Los hiperparámetros a variar en cada clasificador fueron: 
- Decision Tree: criterio "gini" o "entropy". 
- KNN: cantidad de vecinos. 
- Random Forest: criterio "gini" o "entropy", bootstrap "True" o "False". 
 
Despues de la aplicación de HalvingGridSearchCV, se obtuvo que la mejor combinación fue: 
- Porcentaje de columnas: 40%. 
- Clasificador: Decision Tree. 
- Criterio clasificador: gini. 
- Rango de n-gramas: (1,2), es decir, frases compuestas por 1 o 2 palabras. 
- Frecuencia mínima de documentos (min_df): 1. Es decir, se aceptan las frases que se presenten al menos 1 vez. 
- Máximo de elementos (max_features): None. Es decir, no limitar la cantidad de elementos presentados resulta en un mejor desempeño. 
 
En cuanto al porcentaje de columnas, puede ser que varias sean redundantes, por ejemplo, 'has_mind_control_resistance' es muy probable que esté altamente correlacionado con 'has_telepathy_resistance'. Por otro lado, otros atributos como "edad" o "peso" puede que no aporten mucho a la clasificación entre héroe o villano. Por esto, el porcentaje obtenido es esperable. 
 
También, tiene sentido que Decision Tree sea el mejor clasificador, ya que las columnas del DF, al ser booleanas, se incorporan bien en el algoritmo de este modelo. Además, en la misma línea, es posible que hayan atributos que sean muy característicos de héroes o villanos, por ejemplo 'has_mind_control' es probable que sea una variable importante para decidir si un personaje es villano. Por esto, utilizar un árbol de decisión es lo óptimo.  
 
La razón de que RandomForestClassifier no tuvo el mejor desempeño, a pesar de ser un modelo más refinado que Decision Tree, puede ser que la aleatoriedad de los muestreos haya influido negativamente en los entrenamientos de los árboles, provocando que las votaciones para las clasificaciones no sean las más precisas. En el caso de KNN, este clasificador está dirigido a Datasets que posean más variables numéricas, y dado que en este caso la predominancia es de booleanos, se encuentra en desventaja frente a los modelos en base a árboles de decisión. 
 
En cuanto al resto de parámetros e hiperparámetros, se encuentra dentro de lo esperable. Por ejemplo, tiene sentido que el rango de n-gramas sea 1 a 2 ya que, por un lado, son raros los n-gramas de 3, por lo que no se aporta información, y solo considerar las palabras unitarias hace que se pierda información. Con la frecuencia de documentos, es esperable también ya que en el idioma, es muy frecuente que las palabras solo se presenten 1 vez, por lo que limitar a una mayor frecuencia, como "5", haría perder información. Se podría ver que ocurre al limitar con frecuencias de 2 o 3 en ese caso. Finalmente, es entendible que en el máximo de elementos sea mejor evaluado que no se limite, ya que corresponde a una limitante que podría dejar afuera palabras que todavía aporten información. En este caso, con mayores recursos de tiempo y computacionales, podrían probarse más variaciones a este parámetro a fin de optimizar.
```

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [126]:
df_comics_no_label_safe = df_comics_no_label[~df_comics_no_label["history_text"].isna()]
df_comics_no_label_safe

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
6,46,A.M.A.Z.O. (CW),NaN,NaN,22,The Anti Meta-human Adaptive Zootomic Organism...,"With its metallic body, it easily overpowered ...",90,100,100,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1368,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
80,1396,Wendigo,Wendigo,NaN,6,The Wendigo are the result of an ancient curse...,NaN,65,80,75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
81,1431,XS (CW),Nora West-Allen,NaN,19,"According to Eobard Thawne, the daughter of Ba...",Nora possesses a natural connection to the Spe...,90,55,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
82,1440,Yondu (MCU),Yondu,NaN,7,"Yondu Udonta was a Centaurian, leader of a key...","Yondu Udonta was a member of the Centaurians, ...",85,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [136]:
y_pred = best_pipe.predict(df_comics_no_label_safe[df_comics_no_label_safe["name"].isin("Vergil  Gorilla Girl  Batcow".split())])
y_pred

array(['Bad', 'Neutral', 'Bad'], dtype=object)

**Vergil**: Bad

In [139]:
df_comics_no_label_safe[df_comics_no_label_safe["name"] == "Vergil"].iloc[0]["history_text"]

"Vergil, later also known as Nelo Angelo, is one of the main characters in Devil May Cry series, acting as the main antagonist of Devil May Cry 3: Dante's Awakening. He is the twin brother and father of the protagonists, Dante and Nero respectively, and seems designed to be their opposites. Despite being the antagonist, Vergil rarely acted beside Dante."

La respuesta es correcta, ya que de la historia se menciona que este personaje es el principal antagonista de Devil May Cry 3

**Gorilla Girl**: Neutral

In [140]:
df_comics_no_label_safe[df_comics_no_label_safe["name"] == "Gorilla Girl"].iloc[0]["history_text"]

'A carnival performer with the ability to turn into a gorilla, Fahnbullah Eddy and the rest of her fellow performers had their souls stolen by the magician Moondark and turned into his mindless slaves. The carnival was freed from Moondark by Spider-Man and Ghost Rider and their souls returned. Sometime after the incident with Moondark, Eddy and her fellow carnival performers Six and Muck briefly operated as a superhero team called the Freaks. In their only known outing as such, the team saved a doctor from being kidnapped by the villains Hammer and Anvil.'

La respuesta es posiblemente correcta, ya que de la historia se menciona que este personaje participo brevemente de un equipo de superhéroes. Pero no hace enfasis, si es un villano ni un heroe.

**Batcow**: Bad

In [141]:
df_comics_no_label_safe[df_comics_no_label_safe["name"] == "Batcow"].iloc[0]["history_text"]

'Bat-Cow was originally a cow that was found by Robin during an investigation on a slaughterhouse, which the cow was sent to. Robin wanted to keep the cow and named it Bat-Cow. Batman decided they would keep the cow. During a later investigation, Batman discovered it had mind-controlling radiation inside of it, leading him to attempt to get rid of the cow. Robin, however would not allow him to hurt the cow. After the events of Robin\'s death, Bat-Cow and Robin\'s dog felt a great loss in morale. Alfred Pennyworth and Nightwing had to take care of the cow. Later after Nightwing\'s disappearance, Alfred was the only caretaker of Bat-Cow. Bat-Cow, complete with cape, spent time on a Wayne Enterprises-owned dairy farm. It was there the cow encountered the cosmic-powered "Forever People." One of them seemed to gain guidance from Bat-Cow\'s actions, namely when Bat-Cow gave one of the Forever People a scarecrow\'s hat. Bat-Cow formed a friendship with Titus, a dog and household cat.'

La respuesta es posiblemente incorrecta, ya que de la historia no se menciona que el personaje sea villano ni un heroe. Una respuesta correcta, debiese ser Neutral.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>